In [7]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import time
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding
import plotly.express as px
import plotly.graph_objs as go
import ipywidgets as widgets
import os
from sklearn import preprocessing
import io
from PIL import Image
import librosa
import sys
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio
import os, random

min_max_scaler = preprocessing.MinMaxScaler()
n_neighbors = 10
FREQ_BINS=30
TIME_STEPS=20
SR=44100

def plotly_able_df(df,num_d=2):
    
    df["feats"] = df["feats"].apply(literal_eval)
    X = df["feats"]
    #convert series of arrays into a numpy array
    X = pd.DataFrame(X.to_list()).to_numpy()
    X = min_max_scaler.fit_transform(X)
    time_start = time.time()
#     tsne = TSNE(n_components=num_d, random_state=0, perplexity=100, verbose=1)
    tsne = Isomap(n_neighbors, n_components=num_d)
#     tsne = LocallyLinearEmbedding(n_neighbors,n_components=num_d)
#     tsne = SpectralEmbedding(n_neighbors=10,n_components?=num_d,random_state=1)
    X_2d = tsne.fit_transform(X)
    
    if num_d==2:
        df2=pd.concat([df["path"],df["primary"],df["secondary"],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1])],axis=1)
        df2.columns=["path","primary","secondary","D1","D2"]
    if num_d==3:
        df2=pd.concat([df["path"],df["primary"],df["secondary"],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1]),pd.Series(X_2d[:,2])],axis=1)
        df2.columns=["path","primary","secondary","D1","D2","D3"]
    
    df2.primary = df2.primary.astype('str')
    return df2

class interactive_graph():
    def hover_fn(self,trace, points, state):
        if points.point_inds:
            ind = points.point_inds[0]
            drmName=trace.customdata[ind][0][:]
            filename=os.getcwd()+"/"+drmName
            with open(filename,'rb') as f:
                audio_data = f.read()
            self.aud.value=audio_data
            self.hover_data.value = str(drmName)+"\n"
#             self.audioImg.value=self.audDisplay(filename)
            
#         return bufD
    def __init__(self,df,grouping_by="primary",title="",num_d=2,sym_size=10,color=px.colors.qualitative.Vivid):
        if num_d==2:
            p = px.scatter(df, x="D1",y="D2",symbol="secondary",color=grouping_by,hover_data=["path"],color_discrete_sequence=color)
        elif num_d==3:
            p = px.scatter_3d(df, x="D1",y="D2",z="D3",color=grouping_by,hover_data=["path"],symbol="primary",color_discrete_sequence=color)
        for trace in p.data:
            trace.update(hoverinfo="none",hovertemplate= '')
        p.update_traces(marker=dict(size=sym_size,
                              line=dict(width=0,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

        self.hover_data = widgets.Textarea()  
        #audio and img widgets
        self.aud=widgets.Audio(autoplay=True,loop=False,embedding=True)
        layout = go.Layout(hovermode=False,)
        self.fig  = go.FigureWidget(p)
        self.fig.update_layout(scene = dict(
                    camera=dict(eye=dict(x=-1, y=-1, z=0)),
                    aspectmode="cube",),
                    margin=dict( r=0, l=0, b=0, t=0))
                    
        for f in self.fig.data:
            f.on_hover(self.hover_fn)        


In [8]:
meta = pd.read_csv("few_shot_vfy_zenodo/meta/4shot_opennessmiddle.csv")
meta.columns = ["path","primary","fold","secondary"]
meta["path"] = "few_shot_vfy_zenodo/" + meta["path"] 
embeddings = pd.read_csv("few_shot_vfy.csv",header=None,)

In [14]:
X = embeddings.drop(0,axis=1).to_numpy()
X = min_max_scaler.fit_transform(X)
time_start = time.time()
tsne = TSNE(n_components=2, random_state=0, perplexity=100, verbose=1)
# tsne = Isomap(n_neighbors, n_components=2)
#     tsne = LocallyLinearEmbedding(n_neighbors,n_components=num_d)
#     tsne = SpectralEmbedding(n_neighbors=10,n_components?=num_d,random_state=1)
X_2d = tsne.fit_transform(X)

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1360 samples in 0.002s...
[t-SNE] Computed neighbors for 1360 samples in 0.104s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1360
[t-SNE] Computed conditional probabilities for sample 1360 / 1360
[t-SNE] Mean sigma: 5.284923
[t-SNE] KL divergence after 250 iterations with early exaggeration: 52.674782
[t-SNE] KL divergence after 1000 iterations: 0.154394


In [17]:
tsne = pd.concat([embeddings[0],pd.Series(X_2d[:,0]),pd.Series(X_2d[:,1]),],axis=1)
tsne.columns = ["path","D1","D2"]
plottable_df = meta.merge(tsne,on="path")
plottable_df

,path,primary,fold,secondary,D1,D2
0,few_shot_vfy_zenodo/pattern_sounds/pattern_01/...,pattern_01,1,0,16.115044,-13.743118
1,few_shot_vfy_zenodo/pattern_sounds/pattern_01/...,pattern_01,1,0,16.192604,-13.720689
2,few_shot_vfy_zenodo/pattern_sounds/pattern_01/...,pattern_01,1,0,16.665762,-13.505581
3,few_shot_vfy_zenodo/pattern_sounds/pattern_01/...,pattern_01,1,0,16.512930,-13.539200
4,few_shot_vfy_zenodo/pattern_sounds/pattern_01/...,pattern_01,2,0,16.480507,-13.554491
...,...,...,...,...,...,...
1355,few_shot_vfy_zenodo/unwanted_sounds/water/wate...,unknown,11,8,-21.852514,-0.543905
1356,few_shot_vfy_zenodo/unwanted_sounds/water/wate...,unknown,11,8,-21.652384,-0.238652
1357,few_shot_vfy_zenodo/unwanted_sounds/water/wate...,unknown,11,8,-21.757942,-1.832773
1358,few_shot_vfy_zenodo/unwanted_sounds/water/wate...,unknown,11,8,-21.577843,-1.643922


In [18]:
ig=interactive_graph(plottable_df,grouping_by="primary",title="8 dim embedding",num_d=2,sym_size=10,color=px.colors.qualitative.Alphabet,)
display(ig.fig,widgets.HBox([ig.hover_data,ig.aud,]))


FigureWidget({
    'data': [{'customdata': array([['few_shot_vfy_zenodo/pattern_sounds/pattern_01/pattern_01_0…

In [19]:
import plotly.graph_objects as go
fig = go.FigureWidget(data=go.Bar(y=[2, 3, 1]))
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': '690b1a2a-e649-421e-80b9-0ccbe57eb797', 'y': [2, 3, 1]}],
 …